# HOMEWORK 3

In [1]:
!pip -q install controlSBML
import controlSBML as ctl
import control
from controlSBML.util import makeSimulationTimes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
import tellurium as te
import sympy
print("controlSBML version: " + ctl.__version__)

controlSBML version: 0.2.13


<img src="https://github.com/joseph-hellerstein/advanced-controls-lectures/blob/main/Lecture_13_14-Closed-Loop-Systems/Closed-Loop-System.png?raw=true"
     alt="Markdown Monster icon" width=600pt
     style="float: left; margin-right: 10px;" />

In this homework, you will design a closed loop system for controlling the mTOR system as modelled in BioModels 823.
1. (4 pt) Implement the function ``makeTestbed``, a "factory" for creating an evaluation testbed a control designs.
    1. Develop a function ``makeTestbed`` that returns a ``control.Interconnect`` object for the testbed.
        * Inputs: name, model_reference (antimony string, url), kp, ki, kd, kf, noise_amp, noise_frq, disturbance_amp, disturbance_frq
        * Outputs: control.Interconnect
    1. Develop a function ``evaluateTestbed`` that plots the output of the closed loop system (y(t) + n(t))
1. (3 pt) Develop a function ``designClosedLoopSystem`` that searches for $k_P$, $k_I$, $k_D$, $k_F$ such that:
    1. All poles are less than 0:
    1. The DC gain of $H_{RY} \approx 1$;
    1. The DC gain for $H_NY$, $H_DY$ is less than 0.1.
1. (2 pt) Evaluate candidate design parameters from (2) on the testbed in (1) by showing plots of step response.
1. (1 pt) Discusses issues/challenges that you encountered in developing (2) and going from (2) to (3).

Other considerations
* What happens as the reference in put is changed?

# Q1. ``evaluateDesign``

In [ ]:
def evaluateDesign(model_reference, kp=1000, ki=2, kd=0, kf=None, 
                   noise_amp=0, noise_frq=1, disturbance_amp=0, disturbance_frq=1,
                   step_size=1, end_time=100, is_plot=True):
    """
    Create an evaluation test and plots the step response.
    
    Parameters
    ----------
    name: str
        Name of the resulting Interconncted system
    model_reference: str
        antimony string or file path or URL
    kp: float
    ki: float
    kd: float
    kf: float
    step_size: float
    
    Returns
    -------
    control.TransferFunction
        Transfer function from input to output
    """
    ctlsb = ctl.ControlSBML(model_reference, 
                input_names=["pAkt"], output_names=["pDEPTOR"])
    siso = ctl.SISOClosedLoopSystem(ctlsb)
    closed_loop_outputs=["entry.out", "sum_R_F.out", 
          "sum_U_D.out", "exit.out"]
    siso.makeClosedLoopSystem("cl_sys", kp=kp, ki=ki, kd=kd, kf=kf,
        noise_amp=noise_amp, noise_frq=noise_frq,
        disturbance_amp=disturbance_amp, disturbance_frq=disturbance_frq,
        closed_loop_outputs=closed_loop_outputs)
    ts = siso.makeStepResponse(time=1, step_size=1, end_time=end_time,
      points_per_time=2)
    ts.columns = ["input", "e(t)", "system.in", "output"]
    if is_plot:
        ctl.plotOneTS(ts, ylim=[-2, 2], figsize=(10, 5))
    # Construct the transfer function
    sys = siso.closed_loop_system.linearize(0, u0=0)
    tf_vec =  control.ss2tf(sys)
    num = tf.num[0][0]
    den = tf.den[0][0]
    return control.TransferFunction(num, den)
    
# Tests
model_reference = "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000823.2?filename=Varusai2018.xml"
tf = evaluateDesign(model_reference, ki=0, end_time=20)

In [ ]:
tf

# Q2. ``designClosedLoopSystem``

# Q3. Design Evaluation

# Q4. Discussion